In [3]:
import pandas as py
import numpy as np
from sklearn.model_selection import train_test_split
import pickle
from sklearn.model_selection import RandomizedSearchCV
from xgboost.sklearn import XGBRegressor
from scipy.stats import randint, uniform
df = py.read_csv(r'D:\ML\BTP\preprocessing\final_batsmen.csv')

In [4]:
with open('striker_encoder.pkl', 'rb') as file:
    st = pickle.load(file)
with open('venue_encoder.pkl', 'rb') as file:
    vn = pickle.load(file)
with open('bowling_team_encoder.pkl', 'rb') as file:
    op = pickle.load(file)
df['striker'] = st.fit_transform(df['striker'])
df['venue'] = vn.fit_transform(df['venue'])
df['bowling_team'] = op.fit_transform(df['bowling_team'])
y = df.pop('dream 11 score')
X = df
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.2)

In [5]:
xgb1 = XGBRegressor()
parameters = {'nthread': [4],
    'objective': ['reg:squarederror'],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 5, 6, 7, 9],
    'min_child_weight': [4],
    'subsample': uniform(loc=0.5, scale=0.4),
    'colsample_bytree': uniform(loc=0.5, scale=0.4),
    'n_estimators': randint(100, 500),
    'gamma': [0.1, 0.5, 1, 1.5],
    'lambda': [0.01, 0.5, 1, 2],
    'seed': [1]}
xgb_grid = RandomizedSearchCV(xgb1,
                                parameters,
                                cv=4,
                                n_jobs=4,
                                verbose=True,
                                n_iter=100)

In [6]:
xgb_grid.fit(X_train, y_train)

Fitting 4 folds for each of 100 candidates, totalling 400 fits


RandomizedSearchCV(cv=4,
                   estimator=XGBRegressor(base_score=None, booster=None,
                                          callbacks=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=None, device=None,
                                          early_stopping_rounds=None,
                                          enable_categorical=False,
                                          eval_metric=None, feature_types=None,
                                          gamma=None, grow_policy=None,
                                          importance_type=None,
                                          interaction_constraints=None,
                                          learning_rate=...
                                        'lambda': [0.01, 0.5, 1, 2],
                                        'learning_rate': [0.01, 0.05, 0.1],
                                        'max_depth': [3, 5, 6, 7, 9],
                                        'min_child_weight': [4],
                                        'n_estimators': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x0000023B6AAB3400>,
                                        'nthread': [4],
                                        'objective': ['reg:squarederror'],
                                        'seed': [1],
                                        'subsample': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x0000023B6AAB2D40>},
                   verbose=True)

In [7]:
best_random_grid=xgb_grid.best_estimator_

In [8]:
y_pred = best_random_grid.predict(X_test)

In [9]:
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.metrics import r2_score

In [10]:
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)


r2 = r2_score(y_test, y_pred)
print("r2 is:", r2)

Mean Squared Error: 528.1413585196833
r2 is: 0.4468212349718631


In [11]:
pickle.dump(best_random_grid, open('xgb.pkl', 'wb'))